In [15]:
# NLTK paket
import nltk


# Gramatika s obilježjima
engl. **Feature Based Grammers (FBG)** ... proširenje koncepta definiranja CFG-a! 
* obilježje strukture (engl. feature structures) kao skupovi parova (obilježje,vrijednost)
* NLTK prikazuje u ispisu kao *AVM matrice* (engl. *attribute value matrices*)


# Definiranje obilježja u NLTK

Strukture značajki možemo oblikovati prema AVM matrici.

In [2]:
# skup značajki
# primjer 1
fs1 = nltk.FeatStruct(TENSE='past', NUM='sg') # gl.vrijeme i broj
print(fs1) # ispis prema AVM formatu
# primjer 2
fs1 = nltk.FeatStruct(PER=3, NUM='pl', GND='fem') # lice broj, rod
print(fs1['GND'])
fs1['CASE'] = 'acc' # dodavanje novih značajki
print(fs1)

[ NUM   = 'sg'   ]
[ TENSE = 'past' ]
fem
[ CASE = 'acc' ]
[ GND  = 'fem' ]
[ NUM  = 'pl'  ]
[ PER  = 3     ]


In [5]:
# referenciranje
fs2 = nltk.FeatStruct(POS='N', AGR=fs1) # agreement = podskup značajki
print(fs2)
# pristup kao višedimenzionalnom polju ...

print(fs2['AGR'])
print(fs2['AGR']['PER'])


[       [ CASE = 'acc' ] ]
[ AGR = [ GND  = 'fem' ] ]
[       [ NUM  = 'pl'  ] ]
[       [ PER  = 3     ] ]
[                        ]
[ POS = 'N'              ]
[ CASE = 'acc' ]
[ GND  = 'fem' ]
[ NUM  = 'pl'  ]
[ PER  = 3     ]
3


Značajke možemo definirati koristeći AVM sintaksu ili preko pozicijskih argumenata. 

In [4]:
# definiranje znacajki kao AVM
print(nltk.FeatStruct("[POS='N', AGR=[PER=3, NUM='pl', GND='fem']]"))
# definiranje preko pozicijskih argumenata
fs3 = nltk.FeatStruct(NAME='Lee', TELNO='01 27 86 42 96', AGE=33)
print(fs3)

[       [ GND = 'fem' ] ]
[ AGR = [ NUM = 'pl'  ] ]
[       [ PER = 3     ] ]
[                       ]
[ POS = 'N'             ]
[ AGE   = 33               ]
[ NAME  = 'Lee'            ]
[ TELNO = '01 27 86 42 96' ]


Kompleksnije strukture značajki koriste reference (naziv. *reentrance*)

In [15]:
fs4 = nltk.FeatStruct("""
    [NAME='Lee', ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'],
                          SPOUSE=[NAME='Kim', ADDRESS->(1)]]
""")
print(fs4)

[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


Vizualizacija kao DAG: 

![](dag03.png)

## OPERACIJE: Subsidijarnost i unifikacija

Ideja povezivanja obilježja se može interpretirati u kontekstu putova u usmjerenom acikličkom grafu.
 * **subsidijarnost**: $F_1\sqsubseteq F_2$ ako je informacija iz $F_1$ sadržana u $F_2$.
 * **unifikacija**: $F_1\sqcup F_2=F$ tako da $F_1\sqsubseteq F,F_2\sqsubseteq F$

In [22]:
# unifikacija
fs1 = nltk.FeatStruct(NUMBER=74, STREET='rue Pascal')
fs2 = nltk.FeatStruct(CITY='Paris')
fs1_2 = fs1.unify(fs2)

print(fs1_2)

[ CITY   = 'Paris'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]


Kako izbijeći redundaciju obilježja? 

In [25]:

fs0 = nltk.FeatStruct("""[NAME=Lee,
                           ADDRESS=[NUMBER=74,
                                    STREET='rue Pascal'],
                           SPOUSE= [NAME=Kim,
                                    ADDRESS=[NUMBER=74,
                                             STREET='rue Pascal']]]""")
print(fs0)
print('----------------------------------')


fs1 = nltk.FeatStruct("[SPOUSE = [ADDRESS = [CITY = Paris]]]")
print(fs1)
print('----------------------------------')
print(fs1.unify(fs0))
print('----------------------------------')

# Što ako želimo promijeniti po referenci?
fs2 = nltk.FeatStruct(
    """[NAME=Lee, ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'],
       SPOUSE=[NAME=Kim, ADDRESS->(1)]]"""
)
print(fs1.unify(fs2))

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]
----------------------------------
[ SPOUSE = [ ADDRESS = [ CITY = 'Paris' ] ] ]
----------------------------------
[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [           [ CITY   = 'Paris'      ] ] ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [       

## Gramatika sa strukturama značajki

In [7]:
nltk.download('book_grammars')

[nltk_data] Downloading package book_grammars to
[nltk_data]     C:\Users\Domagoj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping grammars\book_grammars.zip.


True

### Sročnost

Primjeri gramatički ispravnih i neispravnih rečenica:
```
   the dog runs 
  *the dog run 
   the dogs run 
  *the dogs runs
```

U klasičnom CFG morali bi definirati kategorije za sročnost po rodu i broju:

```
S -> NP_SG VP_SG
S -> NP_PL VP_PL
NP_SG -> Det_SG N_SG
NP_PL -> Det_PL N_PL
VP_SG -> V_SG
VP_PL -> V_PL

Det_SG -> 'this'
Det_PL -> 'these'
N_SG -> 'dog'
N_PL -> 'dogs'
V_SG -> 'runs'
V_PL -> 'run'
```


Značajke nam omogućuju da to učinimo na elegantniji način koristeći varijable `?x`.

```
S -> NP[NUM=?n] VP[NUM=?n]
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
VP[NUM=?n] -> V[NUM=?n]

Det[NUM=sg] -> 'this'
Det[NUM=pl] -> 'these'

N[NUM=sg] -> 'dog'
N[NUM=pl] -> 'dogs'
V[NUM=sg] -> 'runs'
V[NUM=pl] -> 'run'

```

koje za rečenicu `these dogs run` daje sljedeću stablo parsiranja:

![](ch09-tree-9.png)


In [24]:
# DEMO
# CFG obogaćena s dodatnim značajkama
nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

tokens = 'Kim likes children'.split()
# ispisi parser prema 'feat0.fcfg' za tokens 
# chart parser
cp = nltk.load_parser('grammars/book_grammars/feat0.fcfg',trace=2)
trees = list(cp.parse(tokens))

tree = trees[0]
tree.draw()



% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'
|.Kim .like.chil.|
Leaf Init Rule:
|[----]    .    .| [0:

# Subkategorizacija

Mogućnost tipiziranja riječi. Leksičke kategorije nose obilježje `SUBCAT` kojem se označava kojoj subkategorizaciji pripadaju. Je li glagol tranzitivan ili ne (`SUBCAT=intran|tran`)? Otvara li riječ prostor za zavisnu rečenicu (`SUBCAT=clause`)?

```
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=intrans, TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=trans, TENSE=?t, NUM=?n] NP
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=clause, TENSE=?t, NUM=?n] SBar

V[SUBCAT=intrans, TENSE=pres, NUM=sg] -> 'disappears' | 'walks'
V[SUBCAT=trans, TENSE=pres, NUM=sg] -> 'sees' | 'likes'
V[SUBCAT=clause, TENSE=pres, NUM=sg] -> 'says' | 'claims'

V[SUBCAT=intrans, TENSE=pres, NUM=pl] -> 'disappear' | 'walk'
V[SUBCAT=trans, TENSE=pres, NUM=pl] -> 'see' | 'like'
V[SUBCAT=clause, TENSE=pres, NUM=pl] -> 'say' | 'claim'

V[SUBCAT=intrans, TENSE=past, NUM=?n] -> 'disappeared' | 'walked'
V[SUBCAT=trans, TENSE=past, NUM=?n] -> 'saw' | 'liked'
V[SUBCAT=clause, TENSE=past, NUM=?n] -> 'said' | 'claimed'

```

U primjeru `You claim that you like children` koje je zavisna (objektna) rečenica, produkcijom 

```
SBar -> Comp S
Comp -> 'that' 
```

možemo prepoznati komplement glagola, koja je zavisna rečenica. 

![](ch09-tree-10.png)


### Pomoćni glagoli, inverzije i dopune

Inverzije rečenica (obrati) pojavljuju se u EN i HR jeziku npr. upitnom obliku:
```
  Do you like children?
 *Like you children?
 
  Never have I seen this dog.
  *Never saw I this dog.
```

Pomoćni glagoli kao *do, can, have, ...* mogu se pojaviti u inverzijama.  Produkcija koje detektira inverziju

```
S[+INV] -> V[+AUX] NP VP
```

U određenim okolnostima riječi koje zahtjevaju komplement, uz određene dopune mogu anulirati to svojstvo.

```
  You like Jody.
 *You like.
```

ako u rečenici postoji riječ *who* onda komplement se može ispustiti.

```
  Kim know who you like.   
```
 
**Kategorija "kose crte" (engl. slash category)**: oblika `Y\XP` je kategorija ili fraza `Y` kojoj nedostaje podkonstituent `XP` (npr. `S/NP` jest `S` bez `NP`).


![](ch09-tree-17.png)



In [21]:
nltk.data.show_cfg('grammars/book_grammars/feat1.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
S[-INV] -> NP VP
S[-INV]/?x -> NP VP/?x
S[-INV] -> NP S/NP
S[-INV] -> Adv[+NEG] S[+INV]
S[+INV] -> V[+AUX] NP VP
S[+INV]/?x -> V[+AUX] NP VP/?x
SBar -> Comp S[-INV]
SBar/?x -> Comp S[-INV]/?x
VP -> V[SUBCAT=intrans, -AUX]
VP -> V[SUBCAT=trans, -AUX] NP
VP/?x -> V[SUBCAT=trans, -AUX] NP/?x
VP -> V[SUBCAT=clause, -AUX] SBar
VP/?x -> V[SUBCAT=clause, -AUX] SBar/?x
VP -> V[+AUX] VP
VP/?x -> V[+AUX] VP/?x
# ###################
# Lexical Productions
# ###################
V[SUBCAT=intrans, -AUX] -> 'walk' | 'sing'
V[SUBCAT=trans, -AUX] -> 'see' | 'like'
V[SUBCAT=clause, -AUX] -> 'say' | 'claim'
V[+AUX] -> 'do' | 'can'
NP[-WH] -> 'you' | 'cats'
NP[+WH] -> 'who'
Adv[+NEG] -> 'rarely' | 'never'
NP/NP ->
Comp -> 'that'


In [25]:
tokens = 'who do you claim that you like'.split()
# 'you claim that you like cats'
# 'rarely do you sing'
cp = nltk.load_parser('grammars/book_grammars/feat1.fcfg')
trees = list(cp.parse(tokens))

tree = trees[0]
tree.draw()
